In [1]:
import pandas as pd
import torch
from SoPred.march.neumf import SoPredModel
from RtPred.march.neumf import RtPredModel


In [3]:
testfile = 'dataset/test.csv'
test_df = pd.read_csv(testfile)

n_users = test_df['reviewerID'].max() + 1
n_items = test_df['asin'].max() + 1

model1_pth = 'models/best_model1.pt'
model2_pth = 'models/best_model2.pt'

model1 = SoPredModel(n_users, n_items)
model1.load_state_dict(torch.load(model1_pth)['model_state_dict'])

model2 = RtPredModel(n_users, n_items)
model2.load_state_dict(torch.load(model2_pth)['model_state_dict'])

/tmp/ipykernel_2134/3389932401.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model1.load_state_dict(torch.load(model1_pth)['model_state_dict'])


RuntimeError: Error(s) in loading state_dict for SoPredModel:
	Missing key(s) in state_dict: "mf_usr_emb.weight", "mf_item_emb.weight", "nn_usr_emb.weight", "nn_item_emb.weight", "neumf.weight", "neumf.bias". 
	Unexpected key(s) in state_dict: "user_embedding.weight", "item_embedding.weight", "fc4.weight", "fc4.bias". 
	size mismatch for fc1.weight: copying a param with shape torch.Size([512, 256]) from checkpoint, the shape in current model is torch.Size([32, 64]).
	size mismatch for fc1.bias: copying a param with shape torch.Size([512]) from checkpoint, the shape in current model is torch.Size([32]).
	size mismatch for fc2.weight: copying a param with shape torch.Size([256, 512]) from checkpoint, the shape in current model is torch.Size([16, 32]).
	size mismatch for fc2.bias: copying a param with shape torch.Size([256]) from checkpoint, the shape in current model is torch.Size([16]).
	size mismatch for fc3.weight: copying a param with shape torch.Size([64, 256]) from checkpoint, the 